<a href="https://colab.research.google.com/github/pavi-ninjaac/Specrom_Analysis/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries
import pandas as pd
import numpy as np
import os
import time
import json


In [3]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3040988 sha256=0048a076ab6e31171dba9e089ec61c0445d370adbf47ba8ab97da5aa992cee0a
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [4]:
import fasttext

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [14]:
# load the data files
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/111778997-2020-12-23.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/395563332-2020-12-24.csv')
print("The size of the dataset 1---->",df1.shape)
print("The size of the dataset 2---->",df2.shape)

The size of the dataset 1----> (49350, 13)
The size of the dataset 2----> (138448, 13)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
#add those to dataframe into one using the pandas append method
data = df1.append(df2 , ignore_index = True)
print("The shape of the whole dataset--->",data.shape)

The shape of the whole dataset---> (187798, 13)


In [17]:
# columns 
columns = data.columns
print(columns)

Index(['author', 'crawl_date', 'crawl_desc', 'crawl_url', 'date',
       'description', 'full_text', 'img_url', 'site_name', 'title', 'topic',
       'url', 'webpage_source'],
      dtype='object')


### Find the Dublicate based on the 'title' column and remve the dublicate (dedublication)

In [23]:
# find the dublicate using the dublicate function in pandas
dublicate = data[data.duplicated('title')]
print("the number of deblicated samples in the dataset-->",len(dublicate))

the number of deblicated samples in the dataset--> 104956


In [66]:
#removing the dublicates from the dataset
data2 = data[~data.duplicated('title')]
print("The shape of the Resultant dataset----->",data2.shape)

The shape of the Resultant dataset-----> (82842, 13)


#### remove null values 

In [67]:
print("The shape of the  dataset before dropna----->",data2.shape)
data2 = data2.dropna(subset = ['full_text'])
print("The shape of the  dataset after dropna----->",data2.shape)
data2.head()

The shape of the  dataset before dropna-----> (82842, 13)
The shape of the  dataset after dropna-----> (36619, 13)


,author,crawl_date,crawl_desc,crawl_url,date,description,full_text,img_url,site_name,title,topic,url,webpage_source
0,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,2018 champion Rob Cross became the highest-pro...,Watch live coverage of the last two sessions ...,https://e2.365dm.com/20/12/1600x900/skysports-...,Sky Sports,"PDC World Darts Championship, 2020/21: Dirk va...",sport,https://www.skysports.com/darts/news/12288/121...,skysports.com
4,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...,westernjournal.com
6,"[""Aja Styles""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,https://static.ffx.io/images/$zoom_0.3431%2C$m...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,brisbanetimes.com.au
8,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Jonny Evans joined Leicester from West Brom in...,Africa Cup of Nations Jonny Evans joined Leic...,https://e0.365dm.com/20/12/768x432/skysports-j...,Sky Sports,Jonny Evans 'close' to signing new Leicester c...,sport,https://www.skysports.com/football/news/11712/...,skysports.com
11,"[""Jake Johnson""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,truthout.org


## Remove Newline charecter in 'full_text'
otherwise make error in fattext language detection
error-----> '  predict processes one line at a time (remove '\n') '

In [73]:
#remove newline charecter 
#helper function
def clean_fulltext(text):
  text = text.lower()
  text = text.replace('\n',' ')
  return text

print(clean_fulltext('hi iam Mehana \n from the new chareter'))

hi iam mehana   from the new chareter


In [74]:
data2['full_text'] = data2.apply(lambda row : clean_fulltext(row['full_text']) , axis = 1)
data2.head(4)

,author,crawl_date,crawl_desc,crawl_url,date,description,full_text,img_url,site_name,title,topic,url,webpage_source
0,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,2018 champion Rob Cross became the highest-pro...,watch live coverage of the last two sessions ...,https://e2.365dm.com/20/12/1600x900/skysports-...,Sky Sports,"PDC World Darts Championship, 2020/21: Dirk va...",sport,https://www.skysports.com/darts/news/12288/121...,skysports.com
4,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The Food and Drug Administration has offered a...,a few examples include the offering of anti-a...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...,westernjournal.com
6,"[""Aja Styles""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Perth mother Clare has found herself mostly co...,perth mother clare* has found herself mostly ...,https://static.ffx.io/images/$zoom_0.3431%2C$m...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,brisbanetimes.com.au
8,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Jonny Evans joined Leicester from West Brom in...,africa cup of nations jonny evans joined leic...,https://e0.365dm.com/20/12/768x432/skysports-j...,Sky Sports,Jonny Evans 'close' to signing new Leicester c...,sport,https://www.skysports.com/football/news/11712/...,skysports.com


### Detect the Language using the fasttext

In [62]:
# need to extract the language on the basis of full_text
# Here I am using the fasttext to detect the language , more words will give more accurate result

import fasttext

pretrained_model_facebook_path = '/content/drive/MyDrive/Colab Notebooks/dataFiles/lid.176.bin' #store some where in your local machine and use it
model = fasttext.load_model(pretrained_model_facebook_path)
#helper function
def lang_detect(text):
  result = model.predict(text,1)
  return result[0][0][-2:]

lang_detect('hello')

'en'

In [75]:
data2['language'] = data2.apply(lambda row : lang_detect(row['full_text']) , axis = 1)
data2.head(5)

,author,crawl_date,crawl_desc,crawl_url,date,description,full_text,img_url,site_name,title,topic,url,webpage_source,language
0,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,2018 champion Rob Cross became the highest-pro...,watch live coverage of the last two sessions ...,https://e2.365dm.com/20/12/1600x900/skysports-...,Sky Sports,"PDC World Darts Championship, 2020/21: Dirk va...",sport,https://www.skysports.com/darts/news/12288/121...,skysports.com,en
4,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The Food and Drug Administration has offered a...,a few examples include the offering of anti-a...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...,westernjournal.com,en
6,"[""Aja Styles""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Perth mother Clare has found herself mostly co...,perth mother clare* has found herself mostly ...,https://static.ffx.io/images/$zoom_0.3431%2C$m...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,brisbanetimes.com.au,en
8,[],2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Jonny Evans joined Leicester from West Brom in...,africa cup of nations jonny evans joined leic...,https://e0.365dm.com/20/12/768x432/skysports-j...,Sky Sports,Jonny Evans 'close' to signing new Leicester c...,sport,https://www.skysports.com/football/news/11712/...,skysports.com,en
11,"[""Jake Johnson""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The billâs gifts to the wealthy underscore t...,in late-night votes just hours after nearly 5...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,truthout.org,en


## Extract only the English lanuage Samples

In [79]:
#keep only the english language samples

data = data2[data2['language'] == 'en']

print("The whole dataset size----->",data2.shape)
print()
print('The English Dataset size--->',data.shape)
print()
print("Other language datas------->",data2.shape[0]-data.shape[0])

The whole dataset size-----> (36619, 14)

The English Dataset size---> (36587, 14)

Other language datas-------> 32


English
